In [12]:
%run purpleair/merge-pa.py --csv bigtable.csv --folder purpleair/sample

In [14]:
%run worldpop/load-wp.py --csv bigtable.csv --folder worldpop/GIS

In [15]:
%run cyclic/build-cyclic.py --csv bigtable.csv --column datetime

In [9]:
import pandas as pd
import numpy as np

df = pd.read_csv("bigtable.csv")
df['time'] = pd.to_datetime(df["datetime"],format="%Y-%m-%d %H:%M:%S %Z")
df['date'] = pd.Series(df.time).dt.strftime("%Y-%m-%d")
df['hour'] = pd.Series(df.time).dt.strftime("%H")

print("nb ground station measurement : {}".format(len(df)))


#2019-01-01T02:40:00
df_meteo = pd.read_csv("weather/weather.csv")
df_meteo['time'] = pd.to_datetime(df_meteo["DATE"],format="%Y-%m-%dT%H:%M:%S")
df_meteo['date'] = pd.Series(df_meteo.time).dt.strftime("%Y-%m-%d")
df_meteo['hour'] = pd.Series(df_meteo.time).dt.strftime("%H")
df_meteo['datehour'] = pd.Series(df_meteo.time).dt.strftime("%Y-%m-%d-%H")

print("nb meteo : {}".format(len(df_meteo)))

df_meteo.head()

    

nb ground station measurement : 105054
nb meteo : 14449


,DATE,Temperature,Wind,Dew,Sky,Visibility,ATM,time,date,hour,datehour
0,2019-01-01T00:00:00,"+0089,1","999,9,C,0000,1","+0078,1","99999,9,9,N","006000,1,9,9","10165,1",2019-01-01 00:00:00,2019-01-01,00,2019-01-01-00
1,2019-01-01T00:11:00,"+0089,5","999,9,C,0000,5","+0078,5","00152,5,M,N","004023,5,N,5","99999,9",2019-01-01 00:11:00,2019-01-01,00,2019-01-01-00
2,2019-01-01T00:17:00,"+0089,5","999,9,C,0000,5","+0078,5","00122,5,M,N","002414,5,N,5","99999,9",2019-01-01 00:17:00,2019-01-01,00,2019-01-01-00
3,2019-01-01T00:22:00,"+0083,5","999,9,C,0000,5","+0067,5","00122,5,M,N","001207,5,N,5","99999,9",2019-01-01 00:22:00,2019-01-01,00,2019-01-01-00
4,2019-01-01T00:47:00,"+0080,5","999,9,C,0000,5","+0070,5","00122,5,M,N","001609,5,N,5","99999,9",2019-01-01 00:47:00,2019-01-01,00,2019-01-01-00


In [10]:

df_meteo=df_meteo.groupby(['datehour']).first().reset_index()

print("nb meteo unique hourly : {}".format(len(df_meteo)))

df_meteo.head()


nb meteo unique hourly : 8759


,datehour,DATE,Temperature,Wind,Dew,Sky,Visibility,ATM,time,date,hour
0,2019-01-01-00,2019-01-01T00:00:00,"+0089,1","999,9,C,0000,1","+0078,1","99999,9,9,N","006000,1,9,9","10165,1",2019-01-01 00:00:00,2019-01-01,00
1,2019-01-01-01,2019-01-01T01:14:00,"+0089,5","999,9,C,0000,5","+0078,5","00091,5,M,N","003219,5,N,5","99999,9",2019-01-01 01:14:00,2019-01-01,01
2,2019-01-01-02,2019-01-01T02:10:00,"+0094,5","200,5,N,0015,5","+0083,5","00091,5,M,N","003219,5,N,5","99999,9",2019-01-01 02:10:00,2019-01-01,02
3,2019-01-01-03,2019-01-01T03:00:00,"+0100,1","220,1,N,0031,1","+0089,1","99999,9,9,N","004800,1,9,9","10127,1",2019-01-01 03:00:00,2019-01-01,03
4,2019-01-01-04,2019-01-01T04:52:00,"+0106,5","200,5,N,0026,5","+0094,5","00091,5,W,N","001609,5,N,5","10106,5",2019-01-01 04:52:00,2019-01-01,04


In [11]:
moments = df[['date', 'hour']].apply(tuple, axis=1).tolist()
moments = list(set(moments))
print("nb moments for stations measures : {}".format(len(moments)))


nb moments for stations measures : 8784


In [12]:
new_df = pd.merge(left=df, right=df_meteo, how='left', 
                  left_on=['date','hour'], right_on=['date','hour'])

print("nb ground station measurement after : {}".format(len(new_df)))
print("nb ground station measurement before : {}".format(len(df)))


new_df = new_df.drop(["time_x", "hour", "time_y"], axis=1)

print("***")

nb ground station measurement after : 105054
nb ground station measurement before : 105054
***


In [13]:
new_df.head()

,datetime,pm25,type,sensor,station_id,x,y,Population,Dist-MRoads,Dist-Setl,...,cos_year,date,datehour,DATE,Temperature,Wind,Dew,Sky,Visibility,ATM
0,2019-05-08 15:00:00 UTC,52.33,PurpleAir,B,14th & S ST NW B (undefined) (38.913805 -77.0...,38.913805,-77.03275,54.04969,0.072,-1.541,...,-0.591261,2019-05-08,2019-05-08-15,2019-05-08T15:00:00,"+0211,1","050,1,N,0036,1","+0161,1","99999,9,9,N","016000,1,9,9","10218,1"
1,2019-05-08 16:00:00 UTC,0.17,PurpleAir,B,14th & S ST NW B (undefined) (38.913805 -77.0...,38.913805,-77.03275,54.04969,0.072,-1.541,...,-0.591261,2019-05-08,2019-05-08-16,2019-05-08T16:52:00,"+0228,5","030,5,N,0041,5","+0167,5","00762,5,M,N","016093,5,N,5","10213,5"
2,2019-05-08 15:00:00 UTC,52.33,PurpleAir,B,14th & S ST NW B (undefined) (38.913805 -77.0...,38.913805,-77.03275,54.04969,0.072,-1.541,...,-0.591261,2019-05-08,2019-05-08-15,2019-05-08T15:00:00,"+0211,1","050,1,N,0036,1","+0161,1","99999,9,9,N","016000,1,9,9","10218,1"
3,2019-05-08 16:00:00 UTC,0.17,PurpleAir,B,14th & S ST NW B (undefined) (38.913805 -77.0...,38.913805,-77.03275,54.04969,0.072,-1.541,...,-0.591261,2019-05-08,2019-05-08-16,2019-05-08T16:52:00,"+0228,5","030,5,N,0041,5","+0167,5","00762,5,M,N","016093,5,N,5","10213,5"
4,2019-05-08 17:00:00 UTC,NaN,PurpleAir,B,14th & S ST NW B (undefined) (38.913805 -77.0...,38.913805,-77.03275,54.04969,0.072,-1.541,...,-0.591261,2019-05-08,2019-05-08-17,2019-05-08T17:52:00,"+0228,5","060,5,N,0057,5","+0167,5","01006,5,M,N","016093,5,N,5","10211,5"


In [2]:
df =new_df

df["Temperature"]=pd.Series(df.Temperature).replace('\+', '', regex=True).replace(',', '.', regex=True).apply(pd.to_numeric, args=('coerce',))
df["Dew"]=pd.Series(df.Dew).replace('\+', '', regex=True).replace(',', '.', regex=True).apply(pd.to_numeric, args=('coerce',))
df["ATM"]=pd.Series(df.ATM).replace('\+', '', regex=True).replace(',', '.', regex=True).apply(pd.to_numeric, args=('coerce',))

# Simplify column names for stations
columns = {"Population":"population", "Dist-MRoads":"dist-mroads", "Dist-Setl":"dist-setl", "Dist-Coast":"dist-coast", "Dist-Forest":"dist-forest", "Slope":"slope", "Elevation":"elevation", "DATE":"date", "Temperature":"TEMP", "Wind":"WIND", "Dew":"DEW", "Sky":"SKY", "Visibility":"VIS"}
df = df.rename(columns=columns)
df = df.replace('14th & S ST NW  B (undefined) (38.913805 -77.03275) Primary 60_minute_average 01_01_2019 01_01_2020.csv', '14th & S ST NW B')
df = df.replace('14th & S ST NW  (outside) (38.913805 -77.03275) Primary 60_minute_average 01_01_2019 01_01_2020.csv', '14th & S ST NW A')
df = df.replace('V Street (outside) (38.918491 -77.037393) Primary 60_minute_average 01_01_2019 01_01_2020.csv', 'V Street')
df = df.replace('McMillan 1 (outside) (38.92185 -77.013271) Primary 60_minute_average 01_01_2019 01_01_2020.csv', 'McMillan 1')
df = df.replace('Courthouse  (outside) (38.88812 -77.088094) Primary 60_minute_average 01_01_2019 01_01_2020.csv', 'Courthouse')
df = df.replace('Cheverly (outside) (38.921633 -76.921768) Primary 60_minute_average 01_01_2019 01_01_2020-9.csv', 'Cheverly 9')
df = df.replace('Cheverly (outside) (38.921633 -76.921768) Primary 60_minute_average 01_01_2019 01_01_2020-1.csv', 'Cheverly 1')
df = df.replace('arlington (outside) (38.900099 -77.081078) Primary 60_minute_average 01_01_2019 01_01_2020-1.csv', 'Arlington')
df = df.replace('13th and E St SE B (undefined) (38.882974 -76.988037) Primary 60_minute_average 01_01_2019 01_01_2020.csv', '13th & E ST SE B')
df = df.replace('13th and E St SE (outside) (38.882974 -76.988037) Primary 60_minute_average 01_01_2019 01_01_2020.csv', '13th & E ST SE A')


NameError: name 'new_df' is not defined

In [31]:
df.head()


,datetime,pm25,type,sensor,station_id,x,y,Population,Dist-MRoads,Dist-Setl,...,cos_year,date,datehour,DATE,Temperature,Wind,Dew,Sky,Visibility,ATM
0,2019-05-08 15:00:00 UTC,52.33,PurpleAir,B,14th & S ST NW B (undefined) (38.913805 -77.0...,38.913805,-77.03275,54.04969,0.072,-1.541,...,-0.591261,2019-05-08,2019-05-08-15,2019-05-08T15:00:00,211.1,"050,1,N,0036,1",161.1,"99999,9,9,N","016000,1,9,9",10218.1
1,2019-05-08 16:00:00 UTC,0.17,PurpleAir,B,14th & S ST NW B (undefined) (38.913805 -77.0...,38.913805,-77.03275,54.04969,0.072,-1.541,...,-0.591261,2019-05-08,2019-05-08-16,2019-05-08T16:52:00,228.5,"030,5,N,0041,5",167.5,"00762,5,M,N","016093,5,N,5",10213.5
2,2019-05-08 15:00:00 UTC,52.33,PurpleAir,B,14th & S ST NW B (undefined) (38.913805 -77.0...,38.913805,-77.03275,54.04969,0.072,-1.541,...,-0.591261,2019-05-08,2019-05-08-15,2019-05-08T15:00:00,211.1,"050,1,N,0036,1",161.1,"99999,9,9,N","016000,1,9,9",10218.1
3,2019-05-08 16:00:00 UTC,0.17,PurpleAir,B,14th & S ST NW B (undefined) (38.913805 -77.0...,38.913805,-77.03275,54.04969,0.072,-1.541,...,-0.591261,2019-05-08,2019-05-08-16,2019-05-08T16:52:00,228.5,"030,5,N,0041,5",167.5,"00762,5,M,N","016093,5,N,5",10213.5
4,2019-05-08 17:00:00 UTC,NaN,PurpleAir,B,14th & S ST NW B (undefined) (38.913805 -77.0...,38.913805,-77.03275,54.04969,0.072,-1.541,...,-0.591261,2019-05-08,2019-05-08-17,2019-05-08T17:52:00,228.5,"060,5,N,0057,5",167.5,"01006,5,M,N","016093,5,N,5",10211.5


In [33]:
df.to_csv("bigtable.csv", index=False)

# @todo

#Explain and transform data to numeric vetor or absolute value

# The rate of horizontal travel of air past a fixed point.
# MIN: 0000 MAX: 0900 UNITS: meters per second SCALING FACTOR: 10 
data = df['WIND'].str.split(pat=",", expand=True)
df['WIND'] = data[3].str.lstrip('0')

# The height above ground level (AGL) of the lowest cloud or obscuring phenomena layer aloft with 5/8 or more summation total sky cover, which may be predominantly opaque, or the vertical visibility into a surface-based obstruction. Unlimited = 22000.
# MIN: 00000 MAX: 22000 UNITS: Meters
data = df['SKY'].str.split(pat=",", expand=True)
df['SKY'] = data[0].str.lstrip('0')

# The horizontal distance at which an object can be seen and identified.
# MIN: 000000 MAX: 160000 UNITS: Meters
data = df['VIS'].str.split(pat=",", expand=True)
df['VIS'] = data[0].str.lstrip('0')